<a href="https://colab.research.google.com/github/ajaygahlot/Image-Captioning-using-Neural-Networks/blob/main/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"agahlot","key":"3842bfe2851da68d85c64148a79c39e0"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d shadabhussain/flickr8k

100% 2.13G/2.13G [00:27<00:00, 42.3MB/s]
100% 2.13G/2.13G [00:27<00:00, 82.1MB/s]


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# importing required modules 
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "flickr8k.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

In [ ]:
filename = "/content/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"

file = open(filename,'r')

doc = file.read()

In [ ]:
# creating dictionary with key as image and tokens(5) as values

descriptions = dict()

for line in doc.split('\n'):
  tokens = line.split()
  if len(tokens)>1:
    image_id = tokens[0]
    image_desc = tokens[1:]
    image_id = image_id.split('.')[0]
  
    image_desc = ' '.join(image_desc)
  
    if image_id not in descriptions:
      descriptions[image_id] = list()
  
    descriptions[image_id].append(image_desc)
    
print(len(descriptions))    
descriptions
  

In [ ]:
# cleaning data ------------- changing all tokens to lowercase and removing special characters

import string

table = str.maketrans('','',string.punctuation)

for key,desc_list in descriptions.items():
  
  for i in range(len(desc_list)):
    
    desc = desc_list[i]
    
    desc = desc.split()
    
    desc = [word.lower() for word in desc]
    
    desc = [word.translate(table) for word in desc]
    
    desc = [word for word in desc if len(word)>1]
    
    desc = [word for word in desc if word.isalpha()]
    
    desc_list[i] = ' '.join(desc)

In [ ]:
# creating a vocabulary for all the word present

vocabulary = set()

for key in descriptions.keys():
  [vocabulary.update(d.split()) for d in descriptions[key]]

print(len(vocabulary))

vocabulary


In [ ]:
# saving description file 

with open('descriptions.txt', 'w') as f:
    for key, value in descriptions.items():
        f.write('%s:%s\n' % (key, value))

In [ ]:
# getting all the images present in training set

filename = "/content/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt"
file = open(filename,'r')
doc = file.read()
train = list()
for line in doc.split('\n'):
  identifier = line.split('.')[0]
  train.append(identifier)

print(len(train))
train
  

In [ ]:
filename = "/content/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt"
file = open(filename,'r')
doc = file.read()
test = list()
for line in doc.split('\n'):
  identifier = line.split('.')[0]
  test.append(identifier)

print(len(test))
print(len(train))

1001
6001


In [ ]:


filename = "/content/descriptions.txt"
file = open(filename,'r')
doc = file.read()

descriptions

In [ ]:
# creating dictionary for training set
train_descriptions = dict()
count=0
for key,value in descriptions.items():
    image_id = key
    if image_id in train:
      
      if image_id not in train_descriptions:
        train_descriptions[image_id] = list()
      
      for temp in descriptions[image_id]:
        desc = 'startseq ' + temp + ' endseq'
        #print(desc)
        #break
        train_descriptions[image_id].append(desc)

print(len(train_descriptions))


6000


In [ ]:
# word which occur atleast 10 times

all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print(len(vocab))

1651


In [ ]:
with open('train_descriptions.txt', 'w') as f:
    for key, value in train_descriptions.items():
        f.write('%s:%s\n' % (key, value))

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model

model = InceptionV3(weights='imagenet')

model_new = Model(model.input, model.layers[-2].output)

Using TensorFlow backend.


96116736/96112376 [==============================] - 2s 0us/step


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3  import preprocess_input

def preprocess(image_path):
  img = load_img(filename, target_size=(299, 299))
  x = img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  return x

In [ ]:
import numpy as np
features = dict()
for name in train:

  if(len(name)>0):
    
    filename = "/content/Flickr_Data/Flickr_Data/Images/" + name  + ".jpg"
    image = preprocess(filename)
    fea_vec = model_new.predict(image)
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    features[name] = fea_vec
    



In [ ]:
print(len(features))

6000


In [ ]:
from pickle import dump
dump(features, open('train_features.pkl', 'wb'))

In [ ]:
features = dict()
for name in test:

  if(len(name)>0):
    
    filename = "/content/Flickr_Data/Flickr_Data/Images/" + name  + ".jpg"
    image = preprocess(filename)
    fea_vec = model_new.predict(image)
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    features[name] = fea_vec
    

In [ ]:
print(len(features))

1000


In [ ]:
from pickle import dump
dump(features, open('test_features.pkl', 'wb'))
    

In [ ]:
# 2nd semester


ixtoword = {}
wordtoix = {}

ix = 1

for w in vocab:
  wordtoix[w] = ix
  ixtoword[ix] = w
  ix += 1 

print(len(ixtoword))

#the maximum length of a caption

def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

def max_length(descriptions):
  lines = to_lines(descriptions)
  return max(len(d.split()) for d in lines)


max_length = max_length(train_descriptions)
print(max_length)




1651
34


In [ ]:
# data generator, intended to be used in a call to model.fit_generator()

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from numpy import array

def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-06-19 17:53:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-19 17:53:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-19 17:53:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# Load Glove vectors
import os

import numpy as np

glove_dir = '/content'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
vocab_size = len(wordtoix) + 1
embedding_dim = 200

embedding_matrix = np.zeros((vocab_size,embedding_dim))

for word,i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector 

In [ ]:
embedding_matrix.shape

(1652, 200)

In [ ]:
from keras import Input
from keras.layers import Dropout,Dense,Embedding,LSTM
from keras.layers.merge import add

# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256,activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size,embedding_dim,mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

#decoder model

decoder1 = add([fe2,se3])
decoder2 = Dense(256,activation='relu')(decoder1)
outputs = Dense(vocab_size,activation='softmax')(decoder2)

# merge the two input models

model = Model(inputs=[inputs1,inputs2],output=outputs)




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
epochs = 10
number_pics_per_batch = 3
steps = len(train_descriptions)#number_pics_per_bath

In [ ]:
from pickle import dump, load

train_features = load(open("/content/train_features.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

train_features

Photos: train=6000


{'2513260012_03d33305cf': array([0.0844795 , 0.09467562, 0.09230019, ..., 0.07690583, 1.4561863 ,
        0.45119092], dtype=float32),
 '2903617548_d3e38d7f88': array([0.17603159, 0.515666  , 0.41329515, ..., 0.65110505, 0.35608774,
        0.97732437], dtype=float32),
 '3338291921_fe7ae0c8f8': array([0.16187215, 0.13512462, 0.04707848, ..., 0.35250127, 0.25436527,
        0.38996595], dtype=float32),
 '488416045_1c6d903fe0': array([0.45007205, 0.07103869, 0.14107424, ..., 0.51038975, 1.1378474 ,
        0.47370905], dtype=float32),
 '2644326817_8f45080b87': array([0.26103926, 0.8520194 , 0.03016775, ..., 0.055092  , 1.0150235 ,
        0.93701875], dtype=float32),
 '218342358_1755a9cce1': array([0.736243  , 0.16550446, 0.12633765, ..., 0.32769346, 0.28599405,
        0.58553267], dtype=float32),
 '2501968935_02f2cd8079': array([0.15158674, 0.0629134 , 1.0291823 , ..., 0.07257841, 1.0983999 ,
        0.99730676], dtype=float32),
 '2699342860_5288e203ea': array([0.30136296, 0.28781295, 

In [ ]:
for i in range(epochs):                                                                                                   
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_batch)                 
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
   

Epoch 1/1
6000/6000 [==============================] - 438s 73ms/step - loss: 3.5827
Epoch 1/1
6000/6000 [==============================] - 442s 74ms/step - loss: 2.9807
Epoch 1/1
6000/6000 [==============================] - 425s 71ms/step - loss: 2.7962
Epoch 1/1
6000/6000 [==============================] - 415s 69ms/step - loss: 2.6894
Epoch 1/1
6000/6000 [==============================] - 429s 71ms/step - loss: 2.6174
Epoch 1/1
6000/6000 [==============================] - 420s 70ms/step - loss: 2.5647
Epoch 1/1
6000/6000 [==============================] - 408s 68ms/step - loss: 2.5252
Epoch 1/1
6000/6000 [==============================] - 412s 69ms/step - loss: 2.4945
Epoch 1/1
6000/6000 [==============================] - 412s 69ms/step - loss: 2.4678
Epoch 1/1
6000/6000 [==============================] - 414s 69ms/step - loss: 2.4460


In [ ]:
model.save('model' + str(i) + '.h5')

In [ ]:
for i in range(epochs):                                                                                                   
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_batch)                 
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

Epoch 1/1
6000/6000 [==============================] - 414s 69ms/step - loss: 2.4279
Epoch 1/1
6000/6000 [==============================] - 408s 68ms/step - loss: 2.4145
Epoch 1/1
6000/6000 [==============================] - 408s 68ms/step - loss: 2.4002
Epoch 1/1
6000/6000 [==============================] - 404s 67ms/step - loss: 2.3883
Epoch 1/1
6000/6000 [==============================] - 406s 68ms/step - loss: 2.3787
Epoch 1/1
4901/6000 [=======================>......] - ETA: 1:14 - loss: 2.3676

In [ ]:
from keras.models import load_model   

# load model
model = load_model('model9.h5')
# summarize model.
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 200)      330400      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
test_features = load(open("/content/test_features.pkl", "rb"))
print('Photos: train=%d' % len(test_features))

Photos: train=1000


In [ ]:
from keras.preprocessing.sequence import pad_sequences

def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final


In [ ]:
def beam_search(image,beam_index=3):
  start = [wordtoix["startseq"]]
  start_word = [[start,0.0]]

  while len(start_word[0][0]) < max_length:
    temp = []
    for s in start_word:
      par_caps = pad_sequences([s[0]], maxlen=max_length,padding='post')
      preds = model.predict([image,par_caps])

      word_preds = np.argsort(preds[0])[-beam_index:]

      for w in word_preds:
        next_cap,prob = s[0][:],s[1]
        next_cap.append(w)
        prob += preds[0][w]
        temp.append([next_cap,prob])
    
    start_word = temp
    start_word = sorted(start_word,reverse=False,key=lambda l: l[1])
    start_word = start_word[-beam_index:]
  
  start_word = start_word[-1][0]
  intermediate_caption = [ixtoword[i] for i in start_word]
  final_caption = []

  for i in intermediate_caption:
    if i!="endseq":
      final_caption.append(i)
    else:
      break

  final_caption = final_caption[1:]

  for val in  final_caption:
  	print(val,end=" ")      



In [ ]:
images = "/content/Flickr_Data/Flickr_Data/Images/"


import matplotlib.pyplot as plt


for z in range(0,999):
  pic = list(test_features.keys())[z]
  image = test_features[pic].reshape((1,2048))
  x=plt.imread(images+pic+'.jpg')
  plt.imshow(x)
  plt.show()
  greedy_caption = greedySearch(image)
  print(greedy_caption)
  #print("beam search: ",end="")
  #beam_search(image)



  print("\n\nRefrences : ")

  '''caption = list(greedy_caption.split(' '))
  text = []

  for val in descriptions[pic]:
  temp = list(val.split(' '))
  text.append(temp)

  print(hypotheses)
  print(references)

  hypotheses = [caption]
  references = [text] 

  print(bleu_score(hypotheses, references))'''

  for val in descriptions[pic]:
    print(val)


In [ ]:
import nltk

def bleu_score(hypotheses, references):
  print(len(hypotheses))
  print(len(references))
  return nltk.translate.bleu_score.corpus_bleu(references, hypotheses)

In [ ]:
hyp1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
         'ensures', 'that', 'the', 'military', 'always',
       'obeys', 'the', 'commands', 'of', 'the', 'party']
ref1a = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
          'ensures', 'that', 'the', 'military', 'will', 'forever',
          'heed', 'Party', 'commands']
ref1b = ['It', 'is', 'the', 'guiding', 'principle', 'which',
          'guarantees', 'the', 'military', 'forces', 'always',
          'being', 'under', 'the', 'command', 'of', 'the', 'Party']
ref1c = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
          'army', 'always', 'to', 'heed', 'the', 'directions',
          'of', 'the', 'party']
hyp2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was',
         'interested', 'in', 'world', 'history']
ref2a = ['he', 'was', 'interested', 'in', 'world', 'history',
          'because', 'he', 'read', 'the', 'book']
list_of_references = [[ref1a, ref1b, ref1c]]
hypotheses = [hyp1]
bleu_score(hypotheses,list_of_references)

1
1


0.5046

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/My\ Drive

In [ ]:
!cp -r /content/model9.h5  /content/drive/My\ Drive

cp: cannot create regular file '/content/drive/My Drive': No such file or directory
